<a href="https://colab.research.google.com/github/Davioliveira1305/Metodos-Numericos-2/blob/main/Tarefa7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Funções Hiperbólicas

In [58]:
import numpy as np

# Hiperbólica seno
def sinh(x):
    return (np.exp(x) - np.exp(-x)) / 2

# Hiperbólica cosseno
def cosh(x):
    return (np.exp(x) + np.exp(-x)) / 2

# Hiperbólica tangente
def tanh(x):
    return sinh(x) / cosh(x)


## Fórmulas de Gauss-Legendre de 2 a 4 pontos

In [59]:
import math
def formulas_gl(f, n, a , b):
  interval = (b - a)/2
  if n == 2:
    # Vetor de pesos referente a fórmula de Gauss-Legendre com dois pontos
    w = [1, 1]
    # x(a1) e x(a2)
    x_a1 = ((a + b)/2) - (((b - a)/2)*math.sqrt(1/3))
    x_a2 = ((a + b)/2) + (((b - a)/2)*math.sqrt(1/3))
    return interval * (f(x_a1)*w[0] + f(x_a2)*w[1])
  elif n == 3:
    # Vetor de pesos referente a fórmula de Gauss-Legendre com 3 pontos
    w = [5/9, 8/9, 5/9]
    # x(a1), x(a2) e x(a3)
    x_a1 = ((a + b)/2) - ((b - a)/2)*math.sqrt(3/5)
    x_a2 = (a + b)/2
    x_a3 = ((a + b)/2) + ((b - a)/2)*math.sqrt(3/5)
    return interval * (f(x_a1)*w[0] + f(x_a2)*w[1] + f(x_a3)*w[2])

  else:
    # Vetor de pesos referente a fórmula de Gauss-Legendre com 3 pontos
    w = [0.6521451549, 0.6521451549, 0.3478548451, 0.3478548451]
    # x(a1), x(a2), x(a3) e x(a4)
    x_a1 = ((a + b)/2) + (((b - a)/2)*math.sqrt((3/7)-(2/7)*math.sqrt(6/5)))
    x_a2 = ((a + b)/2) - (((b - a)/2)*math.sqrt((3/7)-(2/7)*math.sqrt(6/5)))
    x_a3 = ((a + b)/2) + (((b - a)/2)*math.sqrt((3/7)+(2/7)*math.sqrt(6/5)))
    x_a4 = ((a + b)/2) - (((b - a)/2)*math.sqrt((3/7)+(2/7)*math.sqrt(6/5)))
    return interval*(f(x_a1)*w[0] + f(x_a2)*w[1] + f(x_a3)*w[2] + f(x_a4)*w[3])

## Função para particionar o intervalo de integração e para garantir a convergência até o erro especificado.

In [60]:
import warnings, numba
import numpy
warnings.filterwarnings('ignore')
# Lib para acelerar códigos python
@numba.jit
def gauss_legendre(f, a, b, points):
  # Inicializando o erro em 10
  E = 10
  # Vetor para guardar o intervalo de integração
  interval = [a, b]
  # Inicializando com o número de partições igual a 1
  N = 1
  # Inicializando da variável que vai guardas as soluções antigas
  Sol_old = 0
  # Inicializando da variável que vai guardas as soluções novas
  Sol_new = 0
  # Contador de iterações
  i = 0
  while E > 1e-6:
    Sol_new = 0
    # Particiona o intervalo de integração em relação ao número de partições
    partitions = numpy.linspace(interval[0], interval[1], N + 1)
    # Pecorre as partições
    for i in range(len(partitions) - 1):
        a = partitions[i+0]
        b = partitions[i+1]
        # Soma o resulta da interpolação nas diversas partições
        Sol_new += formulas_gl(f,points, a , b)
    # Aumenta o número de partições
    N = N * 2
    # Calcula o erro relativos da solução
    E = abs(1 - Sol_old/Sol_new)
    # Atualiza a solução antiga
    Sol_old = Sol_new
    i += 1
  return Sol_new, i, E

# Função que queremos integrar:
$$\int^{0}_{-2} \frac{dx}{\sqrt{4 - x^{2}}}$$

## Intervalo de Integração

In [61]:
a = -2
b = 0

## Função que será integrada

In [62]:
def func(x):
  return 1/(math.sqrt(4 - x**2))

# Exponencial Simples


## Mudança de variável, utilizando a estratégia de exponencial simples

In [63]:
def mud_variavel(s, a, b):
  return ((a + b)/2) + ((b - a)/2)*tanh(s)

def exp_simples(s, a = a, b = b):
  # Mudança de variável
  x_s = mud_variavel(s, a, b)
  # Derivada em relação a s
  deri = ((b - a)/2) * (1/((cosh(s))**2))
  return func(x_s) * deri


## Definição do melhor intervalo de integração depois da mudança de variável

In [64]:
def convergencia(simples = True):
  c = 1
  new_result = 0
  old_result = 0
  while True:
    if simples:
      new_result = gauss_legendre(exp_simples, -c, c, 4)[0]
      E = abs((new_result - old_result)/new_result)
      old_result = new_result
      if E < 1e-5: break
      c = c + 1
      print(f'Solução = {new_result}, intervalo de integração = [{-c}, {c}]')
    else:
      new_result = gauss_legendre(exp_dupla, -c, c, 4)[0]
      E = abs((new_result - old_result)/new_result)
      old_result = new_result
      if E < 1e-5: break
      c = c + 0.1
      print(f'Solução = {new_result}, intervalo de integração = [{-c}, {c}]')

In [65]:
# Indica que queremos usar a metodologia da exponencial simples
convergencia(True)

Solução = 0.9580559289173585, intervalo de integração = [-2, 2]
Solução = 1.362859746496995, intervalo de integração = [-3, 3]
Solução = 1.4979867576737396, intervalo de integração = [-4, 4]
Solução = 1.5445623715005878, intervalo de integração = [-5, 5]
Solução = 1.5612222131469198, intervalo de integração = [-6, 6]
Solução = 1.567284706648648, intervalo de integração = [-7, 7]
Solução = 1.569505899872518, intervalo de integração = [-8, 8]
Solução = 1.5703217985174749, intervalo de integração = [-9, 9]
Solução = 1.5706217837471892, intervalo de integração = [-10, 10]
Solução = 1.570732119537194, intervalo de integração = [-11, 11]
Solução = 1.570772706743193, intervalo de integração = [-12, 12]


# Exponencial Dupla

## Mudança de variável, utilizando a estratégia de exponencial dupla

In [66]:
def mud_variavel_d(s, a, b):
  return ((a + b)/2) + ((b - a)/2)*tanh((math.pi/2)*sinh(s))

def exp_dupla(s, a = a, b = b):
  # Mudança de variável
  x_s = mud_variavel_d(s, a, b)
  # Derivada em relação a s
  deri = ((b - a)/2) * ((math.pi/2) * ((cosh(s))/((cosh((math.pi/2)*sinh(s)))**2)))
  return func(x_s) * deri

In [67]:
# Indica que queremos utilizar a metodologia da exponencial dupla
convergencia(False)

Solução = 1.325501864500737, intervalo de integração = [-1.1, 1.1]
Solução = 1.3835212912127144, intervalo de integração = [-1.2000000000000002, 1.2000000000000002]
Solução = 1.4305664224282475, intervalo de integração = [-1.3000000000000003, 1.3000000000000003]
Solução = 1.4680470194295563, intervalo de integração = [-1.4000000000000004, 1.4000000000000004]
Solução = 1.4973278593790682, intervalo de integração = [-1.5000000000000004, 1.5000000000000004]
Solução = 1.5196973771593236, intervalo de integração = [-1.6000000000000005, 1.6000000000000005]
Solução = 1.5363518572023354, intervalo de integração = [-1.7000000000000006, 1.7000000000000006]
Solução = 1.5483865285989873, intervalo de integração = [-1.8000000000000007, 1.8000000000000007]
Solução = 1.5567878355736824, intervalo de integração = [-1.9000000000000008, 1.9000000000000008]
Solução = 1.5624246359773484, intervalo de integração = [-2.000000000000001, 2.000000000000001]
Solução = 1.5660392409330068, intervalo de integração